# Fraud Detection By Using Open Source Medicare Data - part 1: pig + sklearn


```python
class author(object):
    
    def __init__(self):
    
        self.name     = "Xinyu (Max) Liu"

        self.email    = "xinyulrsm@gmail.com"

        self.linkedin = "linkedin.com/in/maxliuwebsite"
        
        self.github   = "github.com/maxliu/"
        
        self.create   = "01/16/2016"

        self.addreess = "Waltham, MA 02453"
        
        
```


This topic is inspired by ref-1 and ref-2. Here I will show how do I use the same data resource by different techeniques for data preparation, feature engineering and model fitting. I divide this work to four parts:

+ part-1: use hadoop + pig + sklearn ( this work )

+ part-2: use hive + spark

+ part-3: use hive + R

+ part-4: use flink 


References:

1) http://www.dataiku.com/blog/2015/08/12/Medicare_Fraud.html by Pierre Gutierrez @ Dataiku

    Very interesting feature engineering. 
    
2) http://nbviewer.jupyter.org/github/ofermend/IPython-notebooks/blob/master/blog-part-1.ipynb  by Ofer Mendelevitch 

    This is very good and "classic" tutorial for data science by using hadoop, pig, hive, spark and other techniques. 


## Data Source

1) 2013 Part D Prescriber data:

https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Part-D-Prescriber.html

2) 2013 payment data:

https://www.cms.gov/OpenPayments/Explore-the-Data/Dataset-Downloads.html

3) npi exclusions data:

http://oig.hhs.gov/exclusions/exclusions_list.asp#instruct


## Copy data to hadoop

In [ ]:
%%bash
$HADOOP_HOME/bin/hdfs dfs -mkdir -p /data/
$HADOOP_HOME/bin/hdfs dfs -put /home/max/data/PARTD_PRESCRIBER_PUF_NPI_DRUG_13.tab    /data
$HADOOP_HOME/bin/hdfs dfs -put /home/max/data/OP_DTL_GNRL_PGYR2013_P01152016.csv      /data

## pig script for data preparison

In [ ]:
%%writefile dataPrep.pig

Register '/usr/local/pig/lib/piggybank.jar';

/*partd_raw = load '/data/HealthCare/PARTD_13_part.tab' */

partd_raw = load '/data/HealthCare/PARTD_PRESCRIBER_PUF_NPI_DRUG_13.tab' 
    using org.apache.pig.piggybank.storage.CSVExcelStorage('\t', 'YES_MULTILINE', 'NOCHANGE', 'SKIP_INPUT_HEADER')
    as (

        NPI:	int,
        NPPES_PROVIDER_LAST_ORG_NAME: chararray,
        NPPES_PROVIDER_FIRST_NAME:	chararray,
        NPPES_PROVIDER_CITY:	chararray,
        NPPES_PROVIDER_STATE: chararray,
        SPECIALTY_DESC:	chararray,
        DESCRIPTION_FLAG: chararray,
        DRUG_NAME:	chararray,
        GENERIC_NAME:	chararray,
        BENE_COUNT: float,
        TOTAL_CLAIM_COUNT: float,	
        TOTAL_DAY_SUPPLY: float,	
        TOTAL_DRUG_COST:	float,
        BENE_COUNT_GE65:	float,
        BENE_COUNT_GE65_REDACT_FLAG: chararray,	
        TOTAL_CLAIM_COUNT_GE65:	float,
        GE65_REDACT_FLAG:	chararray,
        TOTAL_DAY_SUPPLY_GE65:	float,
        TOTAL_DRUG_COST_GE65:	float
        ) ;


npi_drugs = group partd_raw by NPI;

npi_drugs_table = foreach npi_drugs {
    specilty  = limit partd_raw.SPECIALTY_DESC 1;
    lastname  =	limit partd_raw.NPPES_PROVIDER_LAST_ORG_NAME 1; 
    firstname = limit partd_raw.NPPES_PROVIDER_FIRST_NAME 1 ;
    city      = limit partd_raw.NPPES_PROVIDER_CITY 1;
    state     = limit partd_raw.NPPES_PROVIDER_STATE 1 ;

    generate 
    group as gp, 

    COUNT(partd_raw), 
    flatten(specilty),
    flatten(lastname) as first_name,
    flatten(firstname) as last_name,
    flatten(city) as city,
    flatten(state) as state,

    MIN(partd_raw.TOTAL_CLAIM_COUNT),
    MAX(partd_raw.TOTAL_CLAIM_COUNT),
    SUM(partd_raw.TOTAL_CLAIM_COUNT),
    MIN(partd_raw.TOTAL_DAY_SUPPLY),
    MAX(partd_raw.TOTAL_DAY_SUPPLY),
    SUM(partd_raw.TOTAL_DAY_SUPPLY),
    MIN(partd_raw.TOTAL_DRUG_COST),
    MAX(partd_raw.TOTAL_DRUG_COST),
    SUM(partd_raw.TOTAL_DRUG_COST) ;
};


payment_raw = load '/data/HealthCare/OP_DTL_GNRL_PGYR2013_P01152016.csv' 
    using org.apache.pig.piggybank.storage.CSVExcelStorage(',', 'YES_MULTILINE', 'NOCHANGE', 'SKIP_INPUT_HEADER')
    as (
        Covered_Recipient_Type:chararray,
        Teaching_Hospital_ID:chararray,
        Teaching_Hospital_Name:chararray,
        Physician_Profile_ID:chararray,
        Physician_First_Name:chararray,
        Physician_Middle_Name:chararray,
        Physician_Last_Name:chararray,
        Physician_Name_Suffix:chararray,
        Recipient_Primary_Business_Street_Address_Line1:chararray,
        Recipient_Primary_Business_Street_Address_Line2:chararray,
        Recipient_City:chararray,
        Recipient_State:chararray,
        Recipient_Zip_Code:chararray,
        Recipient_Country:chararray,
        Recipient_Province:chararray,
        Recipient_Postal_Code:chararray,
        Physician_Primary_Type:chararray,
        Physician_Specialty:chararray,
        Physician_License_State_code1:chararray,
        Physician_License_State_code2:chararray,
        Physician_License_State_code3:chararray,
        Physician_License_State_code4:chararray,
        Physician_License_State_code5:chararray,
        Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name:chararray,
        Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID:chararray,
        Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name:chararray,
        Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State:chararray,
        Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country:chararray,
        Total_Amount_of_Payment_USDollars:float,
        Date_of_Payment:chararray,
        Number_of_Payments_Included_in_Total_Amount:chararray,
        Form_of_Payment_or_Transfer_of_Value:chararray,
        Nature_of_Payment_or_Transfer_of_Value:chararray,
        City_of_Travel:chararray,
        State_of_Travel:chararray,
        Country_of_Travel:chararray,
        Physician_Ownership_Indicator:chararray,
        Third_Party_Payment_Recipient_Indicator:chararray,
        Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value:chararray,
        Charity_Indicator:chararray,
        Third_Party_Equals_Covered_Recipient_Indicator:chararray,
        Contextual_Information:chararray,
        Delay_in_Publication_Indicator:chararray,
        Record_ID:chararray,
        Dispute_Status_for_Publication:chararray,
        Product_Indicator:chararray,
        Name_of_Associated_Covered_Drug_or_Biological1:chararray,
        Name_of_Associated_Covered_Drug_or_Biological2:chararray,
        Name_of_Associated_Covered_Drug_or_Biological3:chararray,
        Name_of_Associated_Covered_Drug_or_Biological4:chararray,
        Name_of_Associated_Covered_Drug_or_Biological5:chararray,
        NDC_of_Associated_Covered_Drug_or_Biological1:chararray,
        NDC_of_Associated_Covered_Drug_or_Biological2:chararray,
        NDC_of_Associated_Covered_Drug_or_Biological3:chararray,
        NDC_of_Associated_Covered_Drug_or_Biological4:chararray,
        NDC_of_Associated_Covered_Drug_or_Biological5:chararray,
        Name_of_Associated_Covered_Device_or_Medical_Supply1:chararray,
        Name_of_Associated_Covered_Device_or_Medical_Supply2:chararray,
        Name_of_Associated_Covered_Device_or_Medical_Supply3:chararray,
        Name_of_Associated_Covered_Device_or_Medical_Supply4:chararray,
        Name_of_Associated_Covered_Device_or_Medical_Supply5:chararray,
        Program_Year:chararray,
        Payment_Publication_Date:chararray
        ) ;


npi_payment = group payment_raw by (Physician_First_Name,Physician_Last_Name,
                                    Recipient_City, Recipient_State);
npi_payment_table = foreach npi_payment {
    generate 
    /*flatten(group),*/
    UPPER(group.Physician_First_Name) as first_name,
    UPPER(group.Physician_Last_Name) as last_name,
    UPPER(group.Recipient_City) as city,
    UPPER(group.Recipient_State) as state,
    COUNT(payment_raw ), 
    SUM(payment_raw .Total_Amount_of_Payment_USDollars)
    ;
};

npi_drugs_payment_table = JOIN npi_drugs_table by (first_name, last_name, city, state) 
              LEFT OUTER, npi_payment_table by  (first_name, last_name, city, state) ;


--this is for testing purpose 
/*rmf /data/to_hadoop/npi_payment_drugs_test.csv*/
/*store npi_drugs_payment_table  into '/data/HealthCare/npi_payment_drugs_test.csv' using PigStorage('\t') ;*/


rmf /data/to_hadoop/npi_payment_drugs_all.csv
store npi_drugs_payment_table  into '/data/HealthCare/npi_payment_drugs_all.csv' using PigStorage('\t') ;
HealthCare/

In [ ]:
%%bash  
pig dataPrep.pig

## Model and Prediction

In [1]:
import os 

import zipfile 

from dbfread import DBF

from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression 
from sklearn import ensemble 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import brier_score_loss, precision_score, recall_score,f1_score, roc_auc_score, accuracy_score 
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.cross_validation import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.feature_extraction import DictVectorizer
from sklearn.cluster import KMeans

import pydoop.hdfs as hdfs 
from pydoop.hdfs import path as hpath 

import pandas as pd

from joblib import Parallel, delayed

import random

import scipy
from scipy.stats import ttest_ind

import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.charts import Bar
from bokeh.io import output_notebook, hplot, vplot

import numpy as np

### Read fraud npi list

In [2]:
dataDir = "/home/hduser/T4/data/healthCare/"

In [3]:
%%bash
cd  /home/hduser/T4/data/healthCare/
ls *.DBF

1401EXCL.DBF
1402EXCL.DBF
1403EXCL.DBF
1404EXCL.DBF
1405EXCL.DBF
1406EXCL.DBF
1407EXCL.DBF
1408EXCL.DBF
1409EXCL.DBF
1410EXCL.DBF
1411EXCL.DBF
1412EXCL.DBF
1501EXCL.DBF
1502EXCL.DBF
1503EXCL.DBF
1504EXCL.DBF
1505EXCL.DBF
1506EXCL.DBF
1507EXCL.DBF
1508EXCL.DBF
1509EXCL.DBF
1510EXCL.DBF
1511EXCL.DBF
1512EXCL.DBF


In [4]:
# read all the npis in the DBF files
npis = []
for fnx in os.listdir(dataDir):
    if fnx.endswith('EXCL.DBF'):
        fn = os.path.join(dataDir, fnx)
        for record in DBF(fn):
            #print(record)
            npi = record['NPI']
            if npi != '0000000000':
                #print npi
                npis.append(int(npi))

In [5]:
# build a DataFrame for 'is_fraud' npis
df_npi_fraud = pd.DataFrame([[npi, 1.0] for npi in npis], columns=['npi', 'is_fraud'])

In [6]:
df_npi_fraud.head()

,npi,is_fraud
0,1487898615,1
1,1073842712,1
2,1427499904,1
3,1770765547,1
4,1477874998,1


# Read Pard_D_13 data from hdfs generated by pig

In [7]:
# Read files from HDFS
# Note that the results generated by pig script is a folder instead of a file
# we need need to conbine all the parts in the folder to a pandas DataFrame

def hdfs_read_csv_from_hdfs(path, sep=',', cols=None, col_types=None):
    files = hdfs.ls(path);
    file_parts = []

    for f in files:
        print f
        print os.path.basename(f)
        if os.path.basename(f).startswith('_'):
            continue
        with hdfs.open(f) as fh:
            file_parts.append(pd.read_csv(fh, sep=sep, header=None, error_bad_lines=False))

    df = pd.concat(file_parts, ignore_index=True)
    if cols is not  None:
        if len(cols) == len(df.columns):
            df.columns = cols
    return df

In [8]:
cols1 = ['npi']
cols2 = ['count','specialty']
cols3 = ['last_name' , 'first_name', 'city', 'state']
cols4 = ['claim_min','claim_max','claim_sum','supply_min','supply_max','supply_sum','drug_min',
'drug_max','drug_sum']
cols5 = ['last_name_1' , 'first_name_1', 'city_1', 'state_1']
cols6 = ["payment_count", "total_payment"]

cols = cols1 + cols2 + cols3 + cols4 + cols5 +cols6

In [9]:

data_path = '/data/HealthCare/npi_payment_drugs_all.csv'

df_partD = hdfs_read_csv_from_hdfs(data_path, sep='\t', cols=cols)

hdfs://pocoyo-1:9001/data/HealthCare/npi_payment_drugs_all.csv/_SUCCESS
_SUCCESS
hdfs://pocoyo-1:9001/data/HealthCare/npi_payment_drugs_all.csv/part-r-00000
part-r-00000


In [10]:
# drop 'last_name' , 'first_name', 'city', and 'state' from DataFrame
# we don't need them in the model
df_partD.drop(cols3 + cols5,axis=1, inplace=True)

In [11]:
df_partD.head()

,npi,count,specialty,claim_min,claim_max,claim_sum,supply_min,supply_max,supply_sum,drug_min,drug_max,drug_sum,payment_count,total_payment
0,1528364486,2,Dentist,14,19,33,176,305,481,97.54,162.96,260.500008,NaN,NaN
1,1437177490,5,Dentist,14,89,188,46,2955,4726,106.12,4346.47,5468.750191,NaN,NaN
2,1154586170,2,Dentist,12,17,29,141,170,311,37.09,119.95,157.039997,NaN,NaN
3,1215042155,9,General Surgery,17,87,522,98,410,2107,84.80,377.74,2120.089966,NaN,NaN
4,1104847011,1,Dentist,11,11,11,78,78,78,59.26,59.26,59.259998,NaN,NaN


# NPI and drug group

In [12]:
cols_npi_drug = ["npi", "drug", "count", "total_claim_count", "total_day_supply", "total_drug_cost"]

# the file below is generated by pig
data_path = '/data/HealthCare/partd_13_npi_drug_all.csv'

npi_drug = hdfs_read_csv_from_hdfs(data_path, sep='\t', cols=cols_npi_drug)

hdfs://pocoyo-1:9001/data/HealthCare/partd_13_npi_drug_all.csv/_SUCCESS
_SUCCESS
hdfs://pocoyo-1:9001/data/HealthCare/partd_13_npi_drug_all.csv/part-r-00000
part-r-00000
hdfs://pocoyo-1:9001/data/HealthCare/partd_13_npi_drug_all.csv/part-r-00001
part-r-00001
hdfs://pocoyo-1:9001/data/HealthCare/partd_13_npi_drug_all.csv/part-r-00002
part-r-00002


In [13]:
npi_drug['total_claim_count'] = npi_drug['total_claim_count'].map(lambda x: np.log10(x + 1.0))

In [14]:
npi_drug['total_day_supply'] = npi_drug['total_day_supply'].map(lambda x: np.log10(x + 1.0))

In [15]:
npi_drug['total_drug_cost'] = npi_drug['total_drug_cost'].map(lambda x: np.log10(x + 1.0))

In [16]:
npi_drug.head()

,npi,drug,count,total_claim_count,total_day_supply,total_drug_cost
0,1003000126,LISINOPRIL,1,1.301030,2.756636,2.005909
1,1003000126,SIMVASTATIN,1,1.255273,2.688420,2.013090
2,1003000126,WARFARIN SODIUM,1,1.079181,2.511883,2.221284
3,1003000142,BACLOFEN,1,1.204120,2.654177,2.148633
4,1003000142,MELOXICAM,1,1.505150,2.924796,2.275749


In [17]:
len(npi_drug)

22025671

# Merge Pard_D data and is_fraud data



In [18]:
df = pd.merge(df_partD, df_npi_fraud, how='left', on='npi')

In [24]:
print len(df_partD), len(df_partD.columns)
print len(df), len(df.columns)
del(df_partD)

808069 14
808076 15


In [25]:
# fill nans by 0
df.fillna(0, inplace=True)

In [19]:
df['total_payment'].describe()

count      144.000000
mean      1154.889236
std       5942.482746
min          8.390000
25%         19.472501
50%         57.969999
75%        145.230002
max      46030.399922
Name: total_payment, dtype: float64

In [26]:
# we found 427 record we could use as fraud
print "numbuer of labled 'is_fraud' data : %d" % (len(df[df['is_fraud']==1]))

numbuer of labled 'is_fraud' data : 427


In [22]:
len(df[df['is_fraud']==1])

427

In [23]:
len(df)

808076

In [ ]:
df.head()

In [24]:
# add three more features
df['claim_max-min'] = df['claim_max']-df['claim_min']
df['supply_max-min'] = df['supply_max']-df['supply_min']
df['drug_max-min'] = df['drug_max']-df['drug_min']

# Feature description

## Divide data to training set and validation set¶

### we weill divide the data to training set and validation set. We will only use training set to select feature to avoid data leak.

In [25]:
#TODO add code for five-fold cross-validation

from sklearn.cross_validation import KFold


ix_ran = df.index.values
random.shuffle(ix_ran)

df_len = len(df)
train_len = int(df_len * 0.8)  # 80% for training

#kf = KFold(df_len, n_folds=5)
#for ix_train, ix_valid in kf:

ix_train = ix_ran[:train_len]
ix_valid = ix_ran[train_len:]

df_train = df.ix[ix_train]
df_valid = df.ix[ix_valid]

print  len(ix_train), len(ix_valid)

646460 161616


In [26]:
npi_drug_w_flag_train= pd.merge(npi_drug, df_train[['npi','is_fraud']], how='inner', on=['npi'])

npi_drug_w_flag_all= pd.merge(npi_drug, df[['npi','is_fraud']], how='inner', on=['npi'])

In [27]:
len(npi_drug_w_flag_train[npi_drug_w_flag_train['is_fraud']==1])

13383

In [28]:
cols = ['total_claim_count' , 'total_day_supply' , 'total_drug_cost' ]

In [29]:
# get unique drug names
drugs = set([ drugx for drugx in npi_drug_w_flag_train['drug'].values if isinstance(drugx, str)])
print len(drugs)

1541


In [30]:
print "Total records in npi_drug train set : ", len(npi_drug_w_flag_train)
print "is_fraud recodes :  ", len(npi_drug_w_flag_train[npi_drug_w_flag_train['is_fraud']==1])
npi_drug_w_flag_train.head()

Total records in npi_drug train set :  17600514
is_fraud recodes :   13383


,npi,drug,count,total_claim_count,total_day_supply,total_drug_cost,is_fraud
0,1003000142,BACLOFEN,1,1.204120,2.654177,2.148633,0
1,1003000142,MELOXICAM,1,1.505150,2.924796,2.275749,0
2,1003000142,GABAPENTIN,1,2.004321,3.504471,3.301093,0
3,1003000142,ACETAMINOPHEN WITH CODEINE,1,1.361728,2.751279,2.387354,0
4,1003000142,OXYCODONE HCL/ACETAMINOPHEN,2,1.623249,3.029789,2.995341,0


In [31]:
ng_train = npi_drug_w_flag_train.groupby(['drug', 'is_fraud'])
ng_all = npi_drug_w_flag_all.groupby(['drug', 'is_fraud'])

In [32]:
ngkeys = ng_train.groups.keys()
print len(ngkeys)
print ngkeys[0:5]

2185
[('HEXACHLOROPHENE', 0.0), ('DORZOLAMIDE/TIMOLOL/PF', 0.0), ('GOSERELIN ACETATE', 0.0), ('VALPROIC ACID (AS SODIUM SALT)', 0.0), ('LAPATINIB DITOSYLATE', 0.0)]


In [33]:
drug_has_both_01 = [drugx for drugx in drugs if ((drugx,0.0) in ngkeys ) & ( (drugx,1.0) in ngkeys)]

In [34]:
#TODO use agg to replace the code below
re_drug_tt = dict()
for drugx in drug_has_both_01:
    for colx in cols:
        fraud_0 = ng_train.get_group((drugx,0.0))[colx].values
        fraud_1 = ng_train.get_group((drugx,1.0))[colx].values
        # print len(fraud_0), len(fraud_1)
        if (len(fraud_0)>2) & (len(fraud_1)>2) :
            tt = ttest_ind(fraud_0, fraud_1)
            re_drug_tt[(drugx, colx)] = tt

In [35]:
p005 = [(key, p) for (key, (t, p)) in re_drug_tt.items() if p <=0.05]  # p = 0.1 or 0.05
print len(p005)

259


In [36]:
from bokeh.charts import BoxPlot,Histogram, output_file, show, hplot,vplot
output_notebook()

Loading BokehJS ...

In [37]:
mm=100
drug_name = p005[mm][0][0]
print drug_name
df_bar = pd.concat([ng_all.get_group((p005[mm][0][0],0.0)), ng_all.get_group((p005[mm][0][0],1.0))])
df_bar.head()

CLOTRIMAZOLE


,npi,drug,count,total_claim_count,total_day_supply,total_drug_cost,is_fraud
48,1003000522,CLOTRIMAZOLE,1,1.146128,2.053078,2.428734,0
472,1003002817,CLOTRIMAZOLE,1,1.579784,3.045714,2.925033,0
1946,1003010117,CLOTRIMAZOLE,1,1.342423,2.149219,2.596652,0
2229,1003010687,CLOTRIMAZOLE,1,1.278754,2.653213,2.558324,0
2928,1003012428,CLOTRIMAZOLE,1,1.255273,2.206826,2.532691,0


In [38]:
box = [
    BoxPlot(df_bar, values=col, label='is_fraud', title= "p-value: " + "%0.2e"%(re_drug_tt[(drug_name, col)][1]), 
               color='is_fraud', plot_width=300, plot_height=500)
       for col in ["total_claim_count", "total_day_supply", "total_drug_cost"]
      ]
#output_file(drug_name + '.html')
print drug_name
show(hplot(*box))

CLOTRIMAZOLE


In [39]:
hist = [Histogram(df_bar, values=col, label='is_fraud',  color='is_fraud')
        for col  in ["total_claim_count", "total_day_supply", "total_drug_cost"]
      ]
show(vplot(*hist))

# Featuer engineering

## Drug score
### select drug and catagory from T-test by p value

### logistic regreassion to for a "drug score" calculation

In [40]:
ng_list = []
new_col_all =[]
for i, p005x in enumerate(p005):
    #if i>4:
    #   break
    drug_name = p005x[0][0]
    cat_name = p005x[0][1] 
    
    new_col = drug_name+'_'+cat_name
    new_col_all.append(new_col)

    ng_0 = ng_all.get_group((drug_name,0.0))[['npi', cat_name]]
    ng_1 = ng_all.get_group((drug_name,1.0))[['npi', cat_name]]

    ng_01 = pd.concat([ng_0, ng_1])
    ng_01.rename(columns={cat_name: new_col}, inplace=True)
    ng_list.append(ng_01)


In [41]:
df.head()

,npi,count,specialty,claim_min,claim_max,claim_sum,supply_min,supply_max,supply_sum,drug_min,drug_max,drug_sum,payment_count,total_payment,is_fraud,claim_max-min,supply_max-min,drug_max-min
66937,1528364486,2,Dentist,14,19,33,176,305,481,97.54,162.96,260.500008,0,0,0,5,129,65.42
224179,1437177490,5,Dentist,14,89,188,46,2955,4726,106.12,4346.47,5468.750191,0,0,0,75,2909,4240.35
551437,1154586170,2,Dentist,12,17,29,141,170,311,37.09,119.95,157.039997,0,0,0,5,29,82.86
321933,1215042155,9,General Surgery,17,87,522,98,410,2107,84.80,377.74,2120.089966,0,0,0,70,312,292.94
144844,1104847011,1,Dentist,11,11,11,78,78,78,59.26,59.26,59.259998,0,0,0,0,0,0.00


In [42]:
npi_col = df[['npi']]

In [43]:
w_npi = []

for n, nx in enumerate(ng_list):
    
    nggx = pd.merge(npi_col, nx.drop_duplicates(['npi']), on='npi', how='left')
    #print n, len(nggx), len(npi_col)
    w_npi.append(nggx)

In [44]:
for wx in w_npi:
    col_n = wx.columns[1]
    df[col_n] = wx[col_n].values

In [45]:
wx = w_npi[0]
wx.columns[1]
col_n = wx.columns[1]

In [46]:
len(wx[col_n].values)

808076

In [47]:
del(wx)

In [48]:
df = df.fillna(0)

In [49]:
df_train = df.ix[ix_train]
df_valid = df.ix[ix_valid]

In [50]:
df_train.columns

Index([u'npi', u'count', u'specialty', u'claim_min', u'claim_max',
       u'claim_sum', u'supply_min', u'supply_max', u'supply_sum', u'drug_min',
       ...
       u'HYDROXYZINE HCL_total_drug_cost',
       u'SAXAGLIPTIN HCL_total_claim_count',
       u'PIOGLITAZONE HCL_total_claim_count',
       u'ACETAMINOPHEN WITH CODEINE_total_day_supply',
       u'TIZANIDINE HCL_total_drug_cost', u'ENOXAPARIN SODIUM_total_drug_cost',
       u'DICLOFENAC SODIUM_total_claim_count',
       u'METHOCARBAMOL_total_drug_cost', u'NAPROXEN_total_drug_cost',
       u'PRASUGREL HCL_total_drug_cost'],
      dtype='object', length=277)

In [51]:
df_valid.columns

Index([u'npi', u'count', u'specialty', u'claim_min', u'claim_max',
       u'claim_sum', u'supply_min', u'supply_max', u'supply_sum', u'drug_min',
       ...
       u'HYDROXYZINE HCL_total_drug_cost',
       u'SAXAGLIPTIN HCL_total_claim_count',
       u'PIOGLITAZONE HCL_total_claim_count',
       u'ACETAMINOPHEN WITH CODEINE_total_day_supply',
       u'TIZANIDINE HCL_total_drug_cost', u'ENOXAPARIN SODIUM_total_drug_cost',
       u'DICLOFENAC SODIUM_total_claim_count',
       u'METHOCARBAMOL_total_drug_cost', u'NAPROXEN_total_drug_cost',
       u'PRASUGREL HCL_total_drug_cost'],
      dtype='object', length=277)

In [52]:
# the code below will cause a memory issue, why?

#for ng_list_x in ng_list:
#    df=pd.merge(df, ng_list_x, on='npi', how='left')

In [53]:
X= df_train[new_col_all].values
Y = df_train['is_fraud'].values
clf =  LogisticRegression(C=1e5, class_weight={0:1, 1:4000}, n_jobs=3)
clf.fit(X,Y)
y_p=clf.predict_proba(X)


In [54]:
X = df[new_col_all].values
y_p = clf.predict_proba(X)
df['drug_score'] = y_p[:,1]

df_train = df.ix[ix_train]
df_valid = df.ix[ix_valid]

## speciaty score

In [55]:
spec_dict =[]
spec_fraud_1 = df_train[df_train['is_fraud']==1]['specialty']

In [56]:
from collections import Counter
counts = Counter(spec_fraud_1)

In [57]:
spec_dict =  dict(counts)

In [58]:
spec_dict.get('343',0)

0

In [59]:
df['spec_score'] = df['specialty'].map(lambda x: spec_dict.get(x, 0))

In [60]:
df_train = df.ix[ix_train]
df_valid = df.ix[ix_valid]

# Select features for modeling

In [61]:
numerical_feas_sel = ['count',
                     'claim_min','claim_max', 'claim_sum',
                     'supply_min','supply_max','supply_sum',
                     'drug_min','drug_max','drug_sum',
                     'spec_score','drug_score',
                     'payment_count', 'total_payment']
# 'claim_max-min','supply_max-min','drug_max-min', 
target = 'is_fraud'

# Single classifier

In [62]:
params_0 = {'n_estimators': 100, 'max_depth': 8, 'min_samples_split': 1, 'learning_rate': 0.01}
params_1 = {'n_estimators': 500, 'max_depth': 10, 'min_samples_split': 1, 'class_weight' : {0:1, 1:4000}, 'n_jobs':3}

scaler = StandardScaler()
    
clfs = [
    LogisticRegression(C=1e5,class_weight={0:1, 1:4000}, n_jobs=3),
    
    GaussianNB(),

    ensemble.RandomForestClassifier(**params_1),

    ensemble.ExtraTreesClassifier(**params_1),
    
    ensemble.GradientBoostingClassifier(**params_0)
    
    ]
    
      

In [63]:
X_train = df_train[numerical_feas_sel].values

y_train = df_train['is_fraud'].values
    
X_train = scaler.fit_transform(X_train)

X_valid = df_valid[numerical_feas_sel].values
y_valid = df_valid['is_fraud'].values
X_valid_x= scaler.transform(X_valid)
    

In [64]:
prob_result = []
df_m = []
clfs_fited = []
for clf in clfs:
    print("%s:" %  clf.__class__.__name__)
    clf.fit(X_train,y_train)
    clfs_fited.append(clf)
    y_pred = clf.predict(X_valid_x)
    prob_pos  = clf.predict_proba(X_valid_x)[:, 1]
    prob_result.append(prob_pos)
    m = confusion_matrix(y_valid, y_pred)
    clf_score = brier_score_loss(y_valid, prob_pos, pos_label=y_valid.max())
    print("\tBrier: %1.5f" % (clf_score))
    print("\tPrecision: %1.5f" % precision_score(y_valid, y_pred))
    print("\tRecall: %1.5f" % recall_score(y_valid, y_pred))
    print("\tF1: %1.5f" % f1_score(y_valid, y_pred))
    print("\tauc: %1.5f" % roc_auc_score(y_valid, prob_pos))
    print("\tAccuracy: %1.5f\n" % accuracy_score(y_valid, y_pred))
    df_m.append(
        pd.DataFrame(m, index=['True Negative', 'True Positive'], columns=['Pred. Negative', 'Pred. Positive'])
        )

LogisticRegression:
	Brier: 0.24781
	Precision: 0.00089
	Recall: 0.63736
	F1: 0.00179
	auc: 0.66554
	Accuracy: 0.59912

GaussianNB:
	Brier: 0.98569
	Precision: 0.00056
	Recall: 1.00000
	F1: 0.00113
	auc: 0.59581
	Accuracy: 0.00194

RandomForestClassifier:
	Brier: 0.10541
	Precision: 0.00134
	Recall: 0.42857
	F1: 0.00267
	auc: 0.69008
	Accuracy: 0.81952

ExtraTreesClassifier:
	Brier: 0.24974
	Precision: 0.00079
	Recall: 0.65934
	F1: 0.00159
	auc: 0.65578
	Accuracy: 0.53316

GradientBoostingClassifier:
	Brier: 0.00087
	Precision: 0.00000
	Recall: 0.00000
	F1: 0.00000
	auc: 0.65315
	Accuracy: 0.99912



In [65]:
fpr, tpr, thresholds = roc_curve(y_valid, prob_result[2])

In [66]:
TOOLS = 'pan, wheel_zoom,box_zoom,box_select,crosshair,resize,reset, hover'
p = figure(tools=TOOLS)
p.circle(fpr,tpr, size=4)
p.title = "ROC"
p.xaxis.axis_label  = "FP rate"
p.yaxis.axis_label  = "TP rate"
show(p)

In [67]:
feature_importance = clfs_fited[2].feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)

In [68]:
numerical_feas_sel

['count',
 'claim_min',
 'claim_max',
 'claim_sum',
 'supply_min',
 'supply_max',
 'supply_sum',
 'drug_min',
 'drug_max',
 'drug_sum',
 'spec_score',
 'drug_score',
 'payment_count',
 'total_payment']

In [69]:
feature_importance[sorted_idx]

array([  1.17701921e-06,   5.80449936e-06,   3.37335323e+00,
         4.80236496e+00,   7.17428378e+00,   7.19050148e+00,
         7.98841735e+00,   9.02918617e+00,   9.12886771e+00,
         9.78148893e+00,   1.03543749e+01,   1.09572382e+01,
         1.69792789e+01,   1.00000000e+02])

In [70]:
feas = [numerical_feas_sel[ix] for ix in sorted_idx]
bardata = {"name":feas[::-1], "importance percent":feature_importance[sorted_idx][::-1]}

In [71]:
from bokeh.charts import Bar, output_file, show, hplot
from bokeh.charts.attributes import ColorAttr, CatAttr

In [76]:
bar = Bar(bardata, values="importance percent", label=CatAttr(columns=["name"], sort=False))

In [77]:
show(bar)

## Model Blend